In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer,FreezeCoreTransformer
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit.circuit import QuantumCircuit
from copy import  deepcopy
from qiskit.circuit import Parameter,ParameterVector
import sys
sys.path.append('..')
from AdaptVQE_.My_adapt import QubitAdapt
# Define Molecule
molecule = MoleculeInfo(
    ["Li", "H"], [(0.0, 0.0, 0.0), (0.8*1, 0.0, 0.0)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)

problem = driver.run()
converter = QubitConverter(JordanWignerMapper(), two_qubit_reduction=True)
transformer = FreezeCoreTransformer(
    freeze_core=True,  # how many electrons we have in our active space
    # how many orbitals we have in our active space
    remove_orbitals=[-3, -2]
)
#problem = transformer.transform(problem=problem)

In [4]:
a = QubitAdapt(ES_problem=problem)#UCCSD - 92 terms
len(a.paulistring_withoutz) #320 qubit pool 
# [i for i in a.paulistring_withz ] 

320

In [4]:
[list(i) for i in a.uccop_dict]

[['IIIIIIIIIXZY', 'IIIIIIIIIYZX'],
 ['IIIIIIIIXZZY', 'IIIIIIIIYZZX'],
 ['IIIIIIIXZZZY', 'IIIIIIIYZZZX'],
 ['IIIIIIXZZZZY', 'IIIIIIYZZZZX'],
 ['IIIIIIIIIXYI', 'IIIIIIIIIYXI'],
 ['IIIIIIIIXZYI', 'IIIIIIIIYZXI'],
 ['IIIIIIIXZZYI', 'IIIIIIIYZZXI'],
 ['IIIIIIXZZZYI', 'IIIIIIYZZZXI'],
 ['IIIXZYIIIIII', 'IIIYZXIIIIII'],
 ['IIXZZYIIIIII', 'IIYZZXIIIIII'],
 ['IXZZZYIIIIII', 'IYZZZXIIIIII'],
 ['XZZZZYIIIIII', 'YZZZZXIIIIII'],
 ['IIIXYIIIIIII', 'IIIYXIIIIIII'],
 ['IIXZYIIIIIII', 'IIYZXIIIIIII'],
 ['IXZZYIIIIIII', 'IYZZXIIIIIII'],
 ['XZZZYIIIIIII', 'YZZZXIIIIIII'],
 ['IIIIIIIIYYXY',
  'IIIIIIIIXXXY',
  'IIIIIIIIXYYY',
  'IIIIIIIIYXYY',
  'IIIIIIIIXYXX',
  'IIIIIIIIYXXX',
  'IIIIIIIIYYYX',
  'IIIIIIIIXXYX'],
 ['IIIIIIIYZYXY',
  'IIIIIIIXZXXY',
  'IIIIIIIXZYYY',
  'IIIIIIIYZXYY',
  'IIIIIIIXZYXX',
  'IIIIIIIYZXXX',
  'IIIIIIIYZYYX',
  'IIIIIIIXZXYX'],
 ['IIIIIIYZZYXY',
  'IIIIIIXZZXXY',
  'IIIIIIXZZYYY',
  'IIIIIIYZZXYY',
  'IIIIIIXZZYXX',
  'IIIIIIYZZXXX',
  'IIIIIIYZZYYX',
  'IIIIIIXZZXYX'],
 ['II

In [ ]:
H = converter.convert(problem.hamiltonian.second_q_op())
len(H.to_pauli_op().oplist[0].primitive.to_label())

In [ ]:
paulis = H.to_pauli_op().oplist
paulis_dict = {}
paulis[]

In [ ]:
def pauliOperator2Dict(pauliOperator):
    """
    Converts pauli operator to dict: pauli string as keys and coefficient as value
    Example:
    {I: 0.4, X: 0.6, Z: 0.1, Y: 0.5}.
    
    Args:
        pauliOperator (PauliSumOp): Pauli operator
    Returns:
        dict: Dictionary containing pauli strings as key and coefficient as value
    """
    paulis = pauliOperator.to_pauli_op().oplist
    paulis_dict = {}

    for x in paulis:
        if x.coeff == 0.0:
            continue
        label = x.primitive.to_label()
        coeff = x.coeff
        paulis_dict[label] = coeff

    return paulis_dict

In [ ]:
pauliOperator2Dict(H)

In [ ]:
from qiskit_nature.circuit.library import UCCSD
uccsd = UCCSD(num_particles=problem.num_particles,num_spin_orbitals=problem.num_spatial_orbitals,
              qubit_converter=converter)
uccsd.excitation_ops()

In [ ]:
Paulisum = [converter.convert(i) for i in uccsd.excitation_ops()]
Paulisum[0].primitive[0]

In [ ]:
Paulisum[0].primitive.to_list()[0][0]

In [ ]:
from openvqe.ucc_family import get_energy_ucc

In [ ]:
from qiskit.opflow import X,Y,Z,I
from qiskit.visualization import array_to_latex

In [ ]:
(X^Y^Y^Y).to_matrix()
array_to_latex(1j*(X^Y^Y^Y).to_matrix())

In [ ]:
matrix = (Y^Y^Y^X).to_matrix()
array_to_latex(1j*(Y^Y^Y^X).to_matrix())

In [ ]:
(Y^Y^Y^X)